# Modeling

Nosso principal objetivo na modelagem é realizar uma clusterização dos dados para posteriomente criar um sistema descritivo, ou seja, nossa meta é criar visualizações que possam ser úteis para os médicos.

Assim, mesmo que os nossos dados possuam rótulos devidamente prontos nós tentaremos utilizar a clusterização. Contudo, inicialmente iremos utilizar algoritmos supervisionados para averiguar se os dados na forma como estão conseguem distinguir bem as classes.

## 1. Leitura dos dados

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler,LabelEncoder
from sklearn.feature_selection import RFE, SelectKBest, f_classif
from mlxtend.feature_selection import SequentialFeatureSelector as sfs

In [2]:
#Setando diretorio Data como o atual
os.chdir('../Data')

#Leitura dos dados
# df_parkinson = pd.read_csv('parkinson_normalizado_ss.csv',index_col='name')
# df_parkinson = pd.read_csv('parkinson_normalizado_mm.csv',index_col='name')
df_parkinson = pd.read_csv('parkinson_pca.csv',index_col='name')

#Mapeando dados para conseguir calcular o AUC
df_parkinson['drug'] = df_parkinson['drug'].map({'CBD':1,'placebo':0})

# 2. Modelagem

## 2.1 Classificação

Iremos realizar a modelagem com 3 algoritmos supervisionados:

- Árvore de Decisão
- Random Forest
- Regressão Logística

Como principal métrica de avaliaçã iremos utilizar o AUC (Área Under Curve) por nosso problema ser binário. De qualquer forma também iremos avaliar acurácia, sensibilidade, especificidade e erros para cada pessoa.

Por fim, para garantir o poder generativo dos nossos algoritmos será utilizado uma variação do k-fold.

Gostaríamos de lembrar que será utilizado os dados obtidos após a aplicação do PCA.

### 2.1.1 Sem variáveis categórias

In [3]:
'''Realiza uma avaliação da classificação'''
def evaluate_classifier(X,Y,split=1,type_clf='dt'):
    
    #Pessoas do dataset
    unique_people = X.index.unique()

    #Listas para armazenar os resultados finais
    list_score = []
    list_sens = []
    list_spec = []
    list_auc = []
    list_error = []

    for i in range(500):
        if(split == 1):
            #Sample das pessoas
            people_train = np.random.choice(unique_people,size=11,replace=False)
            people_test = unique_people[~unique_people.isin(people_train)]
    
            #Criando treino e test
            X_train = X.loc[people_train]
            Y_train = Y.loc[people_train]
            X_test = X.loc[people_test]
            Y_test = Y.loc[people_test]
        elif(split == 2):
            X_train,X_test,Y_train,Y_test = train_test_split(X,Y,stratify=Y,test_size=0.4)

        #Criando classificador
        if(type_clf == 'dt'):
            clf = DecisionTreeClassifier().fit(X_train,Y_train)
        elif(type_clf == 'rf'):
            clf = RandomForestClassifier(n_estimators=100).fit(X_train,Y_train)
        else:
            clf = LogisticRegression(solver='liblinear',penalty='l1')

        #Matrix de confusao
        tn, fp, fn, tp = confusion_matrix(Y_test,clf.predict(X_test)).ravel()

        #Sensibilidade
        sens = tp/(tp+fn)
        
        #Especificidade
        spec = tn/(tn+fp)

        #Score
        score = (tn+tp)/(tn+tp+fn+fp)
        
        #AUC
        auc = roc_auc_score(Y_test,clf.predict(X_test))
        
        #Pessoas que erraram
        error = (clf.predict(X_test) == Y_test)
        list_error = np.append(list_error,error[error == False].index.unique().values)
        list_score.append(score)
        list_sens.append(sens)
        list_spec.append(spec)
        list_auc.append(auc)

    print('AUC:',np.nanmean(list_auc))
    print('Score:',np.nanmean(list_score))
    print('Sensibilidade:',np.nanmean(list_sens))
    print('Especificidade:',np.nanmean(list_spec),'\n')
    
    #Calculo do erro por pessoa
    error = pd.Series(list_error).value_counts()
    print(error/error.values.sum())

In [4]:
#Separando dataset em variaveis dependentes e independentes
X = df_parkinson.select_dtypes('float')
Y = df_parkinson['drug']

#Classificacao
evaluate_classifier(X,Y,split=2)

AUC: 0.5169193548387098
Score: 0.5169193548387098
Sensibilidade: 0.5142903225806451
Especificidade: 0.5195483870967742 

person_10    0.047209
person_12    0.046923
person_4     0.046923
person_8     0.046636
person_18    0.046636
person_19    0.046541
person_16    0.046445
person_7     0.046445
person_11    0.046349
person_21    0.046063
person_17    0.046063
person_20    0.046063
person_2     0.045967
person_0     0.045872
person_13    0.045298
person_5     0.045203
person_3     0.043865
person_14    0.043769
person_1     0.043578
person_15    0.043291
person_9     0.042527
person_6     0.042336
dtype: float64


### 2.1.2 Variáveis categórias sem ser dummy

In [5]:
#Separando dataset em variaveis dependentes e independentes
X = df_parkinson.select_dtypes('float').copy()
Y = df_parkinson['drug']

#Adicionando variaveis categoricas
X['measure'] = LabelEncoder().fit_transform(df_parkinson['measure'])
X['evaluate'] = df_parkinson['evaluate']

#Classificacao
evaluate_classifier(X,Y)

AUC: 0.4798961038961039
Score: 0.4798961038961039
Sensibilidade: 0.47553246753246753
Especificidade: 0.48425974025974017 

person_20    0.050638
person_2     0.048998
person_7     0.047359
person_18    0.046995
person_17    0.046812
person_1     0.046084
person_16    0.045537
person_13    0.045173
person_4     0.045173
person_12    0.045173
person_14    0.044991
person_21    0.044627
person_8     0.044627
person_0     0.044627
person_15    0.044627
person_9     0.044627
person_19    0.044627
person_5     0.044627
person_10    0.044444
person_11    0.044080
person_6     0.043534
person_3     0.042623
dtype: float64


### 2.1.3 Variáveis categórias sendo dummy

In [6]:
#Separando dataset em variaveis dependentes e independentes
X = df_parkinson.select_dtypes('float').copy()
Y = df_parkinson['drug']

#Criando dummies
X = pd.concat([X,pd.get_dummies(df_parkinson['measure'],prefix='measure'),pd.get_dummies(df_parkinson['evaluate'],prefix='evaluate')],axis=1)

#Classificacao
evaluate_classifier(X,Y)

AUC: 0.4842987012987013
Score: 0.4842987012987013
Sensibilidade: 0.4781558441558441
Especificidade: 0.4904415584415584 

person_6     0.049353
person_12    0.046986
person_9     0.046986
person_3     0.046986
person_2     0.046986
person_17    0.046440
person_20    0.046440
person_21    0.046258
person_15    0.046075
person_8     0.046075
person_16    0.045893
person_14    0.045711
person_13    0.045711
person_18    0.045347
person_10    0.045347
person_11    0.044983
person_5     0.044983
person_4     0.044436
person_0     0.043708
person_19    0.042433
person_1     0.042069
person_7     0.040794
dtype: float64


### 2.1.3 Váriveis categóricas sendo dummy, porém unidas

In [7]:
#Separando dataset em variaveis dependentes e independentes
X = df_parkinson.select_dtypes('float').copy()

#Criando dummies
X = pd.concat([X,pd.get_dummies(df_parkinson['measure'] + '_' + df_parkinson['evaluate'].map(lambda x: str(x)))],axis=1)

#Classificacao
evaluate_classifier(X,Y)

AUC: 0.5025324675324675
Score: 0.5025324675324675
Sensibilidade: 0.4959480519480519
Especificidade: 0.5091168831168831 

person_9     0.048372
person_18    0.047827
person_5     0.047463
person_12    0.047463
person_14    0.047281
person_8     0.046918
person_13    0.046918
person_21    0.046554
person_16    0.046372
person_19    0.046372
person_2     0.046190
person_4     0.045827
person_10    0.045645
person_6     0.045645
person_15    0.044554
person_1     0.044190
person_7     0.044190
person_0     0.043826
person_20    0.043644
person_11    0.042008
person_3     0.041826
person_17    0.040917
dtype: float64


A utilização de vários tipos de codificação para a variável fase e medida foram tentados por nossas análises mostrarem que existem diferenças dependendo da onde você olha.

Contudo, independente da forma de codificação utilizada não conseguimos bons resultados. Esse fato ocorreu mesmo havendo uma variação dos algoritmos supervisionados.

Acreditamos que essas codificações não derão certo por em cada fase/medida possuir um comportamente diferente. Vamos avaliar essa hipótese na próxima seção.

### 2.1.4 Classificação considerando apenas uma fase e medida

In [8]:
#Filtro selecionado a medida P na avaliação 1 com PC4 e PC5
df_parkinson = df_parkinson.loc[(df_parkinson['measure'] == 'P') & (df_parkinson['evaluate'] == 1),['PC4','PC5','drug']]

In [9]:
#Separando dataset em variaveis dependentes e independentes
X = df_parkinson.loc[:,['PC4','PC5']]
Y = df_parkinson['drug']

#Classificacao
evaluate_classifier(X,Y)

AUC: 0.7508103174603175
Score: 0.7356363636363636
Sensibilidade: 0.7303984126984125
Especificidade: 0.7712222222222223 

person_8     0.149243
person_0     0.128611
person_15    0.087345
person_7     0.077029
person_17    0.072902
person_18    0.066713
person_19    0.061898
person_11    0.061210
person_2     0.057084
person_6     0.047455
person_9     0.039890
person_3     0.039890
person_21    0.024072
person_16    0.022008
person_12    0.017882
person_1     0.017882
person_14    0.014443
person_20    0.011692
person_13    0.002751
dtype: float64


Podemos ver que agora obtivemos bons resultados. Contudo isso foi a custa de uma grande diminuição da quantidade de amostras no dataset.

Salientamos também que esse valor só foi possível de alcançar utilizando o PC4 e PC5. Qualquer outro componente utilizado acaba 'atrapalhando' na classificação.